<a href="https://colab.research.google.com/github/GuangyuNie/MAE_598_Optimization/blob/master/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
import numpy as np
import copy

In [66]:
# obj function f
def f(x):
    return x[0][0]**2 + (x[1][0] - 3)**2

# dfdx
def df(x):
    return np.array([[2*x[0][0]],
                    [2*x[1][0] - 6]]).T

# constraint function g
def g(x):
    return np.array([[x[1][0]**2 - 2*x[0][0]],
                    [(x[1][0] - 1)**2 + 5*x[0][0] - 15]]) 

# dgdx
def dg(x):
    return np.array([[-2, 2*x[1][0]],
                    [5, 2*x[1][0]-2]])

In [67]:
def line_search(x, s, mu, w_old, k):
    t = 0.3
    a = 1
    w = np.zeros((2, 1))

    w[0] = max(abs(mu[0]), 0.5 * (w_old[0] + abs(mu[0])))
    w[1] = max(abs(mu[1]), 0.5 * (w_old[1] + abs(mu[1])))

    dg_da_1 = max(0,np.matmul(dg(x)[0, :], s))
    dg_da_2 = max(0,np.matmul(dg(x)[1, :], s))


    dF_da = np.matmul(df(x), s) + (w[0, :] * dg_da_1 + w[1, :] * dg_da_2)

    def F_a(x, w, a, s):
        g1 = max(0, g(x + a*s)[0, :])
        g2 = max(0, g(x + a*s)[1, :])
        F = f(x + a*s) + (w[0, :] * g1 + w[1, :] * g2)
        return F
 
    phi = lambda x, w, a, t, dF_da: F_a(x, w, 0, 0) + a * t * dF_da

    while phi(x, w, a, t, dF_da) < F_a(x, w, a, s):
        a = 0.8 * a

    return a, w

In [68]:
def solveqp(x, W, df, g, dg):
    A0 = dg(x)
    b0 = g(x)  
    mu0 = np.zeros((b0.shape[0], 1)) 
    mu = [] 
    active_set = []
    terminate = False
    while not terminate:
        if len(active_set) == 0:
            temp = W
            s_and_mu = np.matmul(np.linalg.inv(temp), -df(x).T)
            s = s_and_mu[:2, :]
            mu = []

        if len(active_set) != 0:
            if len(active_set) == 1:
                A = A0[active_set[0], :].reshape(1, -1)
                b = b0[active_set[0], :]
            if len(active_set) == 2:
                A = copy.deepcopy(A0)
                b = copy.deepcopy(b0)
            temp = np.vstack((np.hstack((W, A.T)),
                                np.hstack((A, np.zeros((A.shape[0], A.shape[0]))))))
            s_and_mu = np.matmul(np.linalg.inv(temp), np.vstack((-df(x).T, -b)))
            s = s_and_mu[:2, :]
            mu = s_and_mu[2:, :]
            if len(mu) == 1:
                mu0[0] = s_and_mu[2:3, :]
            if len(mu) == 2:
                mu0[0] = s_and_mu[2:3, :]
                mu0[1] = s_and_mu[3:, :]

        qp_cons = np.round((np.matmul(A0, s.reshape(-1, 1)) + b0))


        if len(mu) == 0:
            mu_check = 1
        elif min(mu) > 0:
            mu_check = 1
        else:
            id_mu = np.argmin(np.array(mu))
            mu.remove(min(mu)) # remove the least violated constraint
            active_set.pop(id_mu)

        if np.max(qp_cons) <= 0:
            if mu_check == 1:
                terminate = True
                return s, mu0
                
        else:
            index = np.argmax(qp_cons)
            active_set.append(index) # add the most violated constraint
            active_set = np.unique(np.array(active_set)).tolist()


In [69]:
def BFGS(W, x, dx, s, mu):
    y_k = (df(x) + np.matmul(mu.T, dg(x))) - (df(x - dx) + np.matmul(mu.T, dg(x - dx)))
    temp = np.matmul(np.matmul(dx.T, W), dx)
    if np.matmul((dx).T, y_k.T) >= 0.2 * np.matmul(np.matmul((dx).T, W), (dx)):
        theta = 1
    else:
        theta = 0.8 * temp / (temp - np.matmul(dx.T, y_k.T))

    dg_k = theta * y_k.T + (1 - theta) * np.matmul(W, dx)
    W_new = W + np.matmul(dg_k, dg_k.T) / np.matmul(dg_k.T, s) - np.matmul(np.matmul(W, s), np.matmul(s.T, W)) / np.matmul(np.matmul(s.T, W), s)

    return W_new
    

In [70]:
x = np.array([[1.], [1.]])
W = np.eye(x.shape[0]) # hessian
w = np.zeros((2,1)) # weights of the merit function for line search
mu_old = np.zeros((x.shape[0], 1))
norm = np.linalg.norm(df(x) + np.matmul(mu_old.T, dg(x)))
error = 1e-4 # termination creterion
k = 0
solution = []
w_old = np.zeros((2, 1))
while norm >= error:

    # solve QP sub-problem
    s, mu_new = solveqp(x, W, df, g, dg)
  
    alpha, w_new = line_search(x, s, mu_old, w_old, k)
    w_old = w_new
    # update current solution of x
    dx = alpha*s
    x = x + dx
    # get new Hessian approx using BFGS
    W = BFGS(W, x, dx, s, mu_new)
    k += 1
    norm = np.linalg.norm(df(x) + np.matmul(mu_new.T, dg(x)))
    mu_old = mu_new
    solution.append(x)

print('solution is x1: {:.4f}, x2: {:.4f}'.format(x[0][0],x[1][0]))

solution is x1: 1.0602, x2: 1.4562
